In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from openfisca_france import CountryTaxBenefitSystem
from openfisca_france.model.base import Famille, FoyerFiscal, Menage
from openfisca_core import periods, parameters
from openfisca_core.model_api import Reform

In [ ]:
import os

In [ ]:
from utils import *

In [ ]:
base_period = "2023-01"

base = CountryTaxBenefitSystem()
base.load_extension('openfisca_france_local')

In [ ]:
full_df = pd.read_excel(os.getenv('DATA_FOLDER') + "mobilite/extrait-Tableau_de_Bord_CTS_Valeurs 012023_ajout_qf_age.xlsx", sheet_name="QRD - Quantités")

In [ ]:
df = full_df[~full_df.Exclu.notna()]
df

In [ ]:
count = len(df)

individu_df = pd.DataFrame({
    'famille_id': list(range(count)),
    'qfrule': df.QF,
    'agerule': df.AGE,
    'taux_incapacite': np.where(df.Titres.str.contains('PMR'), 0.8, 0)
})

In [ ]:
determine_qf(individu_df, qfrules_constant)
determine_age(individu_df)

In [ ]:
famille_df = pd.DataFrame({})
menage_df = pd.DataFrame({
    'eurometropole_strasbourg_tarification_solidaire_transport_eligibilite_geographique': np.ones(count),
})
foyerfiscaux_df = pd.DataFrame({})

individu_df['famille_role_index'] = 0
individu_df['foyer_fiscal_id'] = individu_df.famille_id
individu_df['foyer_fiscal_role_index'] = 0
individu_df['menage_id'] = individu_df.famille_id
individu_df['menage_role_index'] = 0

data = dict(input_data_frame_by_entity = dict(
individu=individu_df,
famille=famille_df,
menage=menage_df,
foyer_fiscal=foyerfiscaux_df))

In [ ]:
scenario = StrasbourgSurveyScenario(tbs=base, data = data)

In [ ]:
res = scenario.simulation.calculate('eurometropole_strasbourg_tarification_transport', base_period)

In [ ]:
res_elig_reduit = scenario.simulation.calculate('eurometropole_strasbourg_tarification_solidaire_transport_eligible_tarif_reduit', base_period)

In [ ]:
res_df = pd.DataFrame(data= {
    'titre_fichier': df.Titres,
    'AGE': df.AGE,
    'QF': df.QF,
    'pmr': individu_df.taux_incapacite > 0,
    'recettes_fichier': df.recettes,
    'quantité_fichier': df.quantité,
    'pu_fichier': df.PU,
    'pu_calc': res,
    'reduit': res_elig_reduit,
    'plein': ~res_elig_reduit
})
#res_df.to_excel('/home/thomas/Nextcloud/CodeursEnLiberte/EMS/mobilite/recalcul_prix_unitaire_titres_2.xlsx')
# pd.pivot(res_df, columns=["QF"], index=["AGE"], values="quantité_fichier")

In [ ]:
count = int(sum(df.quantité))
sample_count = 40
(count, sample_count)

In [ ]:
sample_ids = np.repeat(list(range(sample_count)), count)
sample_qfrule = np.tile(np.repeat(df.QF, df.quantité), sample_count)
sample_individu_df = pd.DataFrame({
    'sample_id': sample_ids,
    'famille_id': list(range(count * sample_count)),
    'qfrule': sample_qfrule,
    'agerule': np.tile(np.repeat(df.AGE, df.quantité), sample_count),
    'taux_incapacite': np.tile(np.repeat(np.where(df.Titres.str.contains('PMR'), 0.8, 0), df.quantité), sample_count)
})

In [ ]:
determine_qf(sample_individu_df, qfrules_alea)

In [ ]:
determine_age(sample_individu_df)

In [ ]:
sample_famille_df = pd.DataFrame({})
sample_menage_df = pd.DataFrame({
    'eurometropole_strasbourg_tarification_solidaire_transport_eligibilite_geographique': np.ones(count * sample_count),
})
sample_foyerfiscaux_df = pd.DataFrame({})

sample_individu_df['famille_role_index'] = 0
sample_individu_df['foyer_fiscal_id'] = sample_individu_df.famille_id
sample_individu_df['foyer_fiscal_role_index'] = 0
sample_individu_df['menage_id'] = sample_individu_df.famille_id
sample_individu_df['menage_role_index'] = 0

sample_data = dict(input_data_frame_by_entity = dict(
individu=sample_individu_df,
famille=sample_famille_df,
menage=sample_menage_df,
foyer_fiscal=sample_foyerfiscaux_df))

In [ ]:
sample_scenario = StrasbourgSurveyScenario(base, data = sample_data)

In [ ]:
sample_res = pd.DataFrame(data= {
    "sample_id": sample_ids,
    "recettes":sample_scenario.simulation.calculate('eurometropole_strasbourg_tarification_transport', base_period)
})
sample_recette = sample_res.groupby(by="sample_id").sum()
sample_recette

In [ ]:
recette_base = sum(res*df.quantité)
sample_recette["pertes"] = sample_recette.recettes-recette_base

In [ ]:
denomb = pd.pivot_table(res_df, index=["QF", "plein"], values="quantité_fichier", aggfunc=sum)
denomb

In [ ]:
merge_bareme_population = denomb.quantité_fichier.cumsum()

In [ ]:
taux_plein = denomb.reset_index()
taux_plein_demi_part = (
    taux_plein[(taux_plein.QF == "TP") * taux_plein.plein].quantité_fichier.iloc[0] +
    2*taux_plein[(taux_plein.QF == "TP") * ~taux_plein.plein].quantité_fichier.iloc[0])
taux_plein_demi_part

In [ ]:
sample_recette['compens'] = -sample_recette.pertes/taux_plein_demi_part

In [ ]:
# demi part abo annuels
reduit_qfmax = 9082 + 3662 + 375 + 275 
taux_plain_qfmax = 9266
taux_plein_annuel_demi_part = taux_plain_qfmax * 2 + reduit_qfmax

In [ ]:
sample_recette['compens_av_annuel'] = -sample_recette.pertes/(taux_plein_demi_part+taux_plein_annuel_demi_part)

In [ ]:
sample_recette[['compens', 'compens_av_annuel']].describe()

In [ ]:
df[["QF", "quantité"]].groupby(by="QF").sum().cumsum()

In [ ]:
static_sample_count = 20
static_sample_ids = np.repeat(list(range(static_sample_count)), count)
static_sample_individu_df = pd.DataFrame({
    'sample_id': static_sample_ids,
    'famille_id': list(range(count * static_sample_count)),
    'qfrule': np.tile(np.repeat(df.QF, df.quantité), static_sample_count),
    'agerule': np.tile(np.repeat(df.AGE, df.quantité), static_sample_count),
    'taux_incapacite': np.tile(np.repeat(np.where(df.Titres.str.contains('PMR'), 0.8, 0), df.quantité), static_sample_count)
})

determine_qf(static_sample_individu_df, qfrules_alea)

In [ ]:
static_sample_individu_df

In [ ]:
rr = static_sample_individu_df.groupby(by="sample_id").eurometropole_strasbourg_tarification_solidaire_transport_quotient_familial.rank(method="first")

In [ ]:
df[["QF", "quantité"]].groupby(by="QF").sum()

In [ ]:
indexes = rr.isin(df[["QF", "quantité"]].groupby(by="QF").sum().cumsum().quantité)
rr_res = pd.DataFrame(data={
    "rank": rr[indexes],
    "value": static_sample_individu_df.eurometropole_strasbourg_tarification_solidaire_transport_quotient_familial[indexes]
})

In [ ]:
rr_res.groupby(by="rank").describe()

In [ ]:
no_reduit_indexes = rr.isin(merge_bareme_population)
no_reduit_rr_res = pd.DataFrame(data={
    "rank": rr[no_reduit_indexes],
    "value": static_sample_individu_df.eurometropole_strasbourg_tarification_solidaire_transport_quotient_familial[no_reduit_indexes]
})

In [ ]:
no_reduit_rr_res.groupby(by="rank").describe()

In [ ]:
from openfisca_core import periods, parameters
from openfisca_core.model_api import Reform

def modify_parameters(local_parameters):
    period = base_period
    reduit = []
    classique = []

    valeurs = [
        # QF CAF
        # [-1, 3.4, 3.4],
        # [167, 5.8, 5.8],
        # [370, 7.3, 7.3],
        # [467, 13.4, 13.4],
        # [582, 13.6, 13.6],
        # [690, 25.7, 25.7],
        # [795, 28, 28],
        # [1021, 56, 56]
        # QF fiscal
        [-1, 3.4, 3.4],
        [50, 5.8, 5.8],
        [233, 7.3, 7.3],
        [319, 13.4, 13.4],
        [423, 13.6, 13.6],
        [520, 25.7, 25.7],
        [615, 28, 28],
        [819, 56, 56],

#        [410, 2.15, 1.80],
#        [510, 2.70, 2.35],
#        [620, 3.25, 2.90],
#        [720, 3.80, 3.45],
    ]

    for tranche in valeurs:
        (qf, r, c) = tranche
        reduit.append(parameters.ParameterScaleBracket(data={
            'threshold': { period: { 'value': qf } },
            'amount': { period: { 'value': r } }
        }))
        classique.append(parameters.ParameterScaleBracket(data={
            'threshold': { period: { 'value': qf } },
            'amount': { period: { 'value': c } }
        }))
    local_parameters.metropoles.strasbourg.tarification_solidaire.bareme_reduit.brackets = reduit
    local_parameters.metropoles.strasbourg.tarification_solidaire.bareme.brackets = classique

    return local_parameters
    
class simulation(Reform):
    name = u"Fusionne les trois premières tranches"

    def apply(self):
        self.modify_parameters(modifier_function = modify_parameters)

        

r = simulation(base)

reform_scenario = StrasbourgSurveyScenario(r, data=sample_data)

res_reform = reform_scenario.simulation.calculate('eurometropole_strasbourg_tarification_transport', base_period)
res_reform

In [ ]:
sample_data['input_data_frame_by_entity']['individu']

In [ ]:
sample_res = pd.DataFrame(data= {
    "sample_id": sample_ids,
    "recettes": res_reform
})
sample_recette = sample_res.groupby(by="sample_id").sum()

In [ ]:
sample_recette["pertes"] = sample_recette.recettes-recette_base

In [ ]:
sample_recette.describe()

In [ ]:
from grist_api import GristDocAPI

In [ ]:
import json

In [ ]:
with open('../grist_info.json') as f:
    grist_info = json.load(f)

api = GristDocAPI(grist_info['docId'], api_key=grist_info['key'], server=grist_info['server'])

In [ ]:
scale_data = api.fetch_table('Baremes_transports')
scale_data

In [ ]:
scenario = "QF_fiscal_bareme_unique"

In [ ]:
getattr(scale_data[0], scenario)

In [ ]:
has_reduit = None
try:
    getattr(scale_data[0], f"{scenario}_reduit")
    has_reduit = True
except AttributeError as e:
    has_reduit = False
has_reduit

In [ ]:
scale_data.sort(key=lambda i: i.QF)

In [ ]:
steps = []

for s in scale_data:
    if getattr(s, scenario):
        steps.append([s.QF-1, getattr(s, f"{scenario}_reduit") if has_reduit else getattr(s, scenario), getattr(s, scenario)])

steps

In [ ]:
from openfisca_core import periods, parameters
from openfisca_core.model_api import Reform

class grist_simulation(Reform):
    name = u"Fusionne les trois premières tranches"

    def __init__(self, tbs, steps):
        self.steps = steps
        super().__init__(tbs)

    def apply(self):
        def modify_parameters_grist(local_parameters):
            period = base_period
            reduit = []
            classique = []

            for tranche in self.steps:
                (qf, r, c) = tranche
                reduit.append(parameters.ParameterScaleBracket(data={
                    'threshold': { period: { 'value': qf } },
                    'amount': { period: { 'value': r } }
                }))
                classique.append(parameters.ParameterScaleBracket(data={
                    'threshold': { period: { 'value': qf } },
                    'amount': { period: { 'value': c } }
                }))
            local_parameters.metropoles.strasbourg.tarification_solidaire.bareme_reduit.brackets = reduit
            local_parameters.metropoles.strasbourg.tarification_solidaire.bareme.brackets = classique
            return local_parameters

        self.modify_parameters(modifier_function = modify_parameters_grist)



In [ ]:
r = grist_simulation(base, steps)

In [ ]:
reform_scenario = StrasbourgSurveyScenario(r, data=sample_data)

res_reform = reform_scenario.simulation.calculate('eurometropole_strasbourg_tarification_transport', base_period)
res_reform

In [ ]:
len(sample_ids)

In [ ]:
len(res_reform)

In [ ]:
df_reform = pd.DataFrame(data={
    'sample_id': sample_ids,
    'qf': reform_scenario.simulation.calculate('eurometropole_strasbourg_tarification_solidaire_transport_quotient_familial', base_period),
    'recettes': reform_scenario.simulation.calculate('eurometropole_strasbourg_tarification_transport', base_period),
    'prix': reform_scenario.simulation.calculate('eurometropole_strasbourg_tarification_transport', base_period)
})
df_reform

In [ ]:
recettes = df_reform[['sample_id', 'recettes']].groupby(by='sample_id').sum().describe()
recettes

In [ ]:
from IPython.display import display, HTML

In [ ]:
display(HTML(recettes.to_html(float_format=lambda x: "{0:,.0f}".format(x).replace(",", " "))))

In [ ]:
pd.pivot_table(df_reform, columns='recettes', index="sample_id", values="prix", aggfunc=len).describe()

In [ ]:
pd.pivot_table(df_reform, columns='recettes', index="sample_id", values="qf", aggfunc=np.max)